# 환경 셋팅

In [1]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


# Bedrock 호출
- 참고: https://docs.anthropic.com/en/api/claude-on-amazon-bedrock

In [2]:
# making requests
import boto3
import json

bedrock = boto3.client(service_name="bedrock-runtime")
body = json.dumps({
  "max_tokens": 256,
  "messages": [{"role": "user", "content": "Hello, world"}],
  "anthropic_version": "bedrock-2023-05-31"
})

response = bedrock.invoke_model(body=body, modelId="anthropic.claude-3-5-sonnet-20240620-v1:0")

response_body = json.loads(response.get("body").read())
print(response_body.get("content"))


[{'type': 'text', 'text': 'Hello! How can I assist you today? Feel free to ask me any questions or let me know if you need help with anything.'}]


# Count input, output tokens

In [3]:
import boto3
import json

bedrock = boto3.client(service_name="bedrock-runtime")
body = json.dumps({
  "max_tokens": 256,
  "messages": [{"role": "user", "content": "Hello, world"}],
  "anthropic_version": "bedrock-2023-05-31"
})

response = bedrock.invoke_model(body=body, modelId="anthropic.claude-3-5-sonnet-20240620-v1:0")

response_body = json.loads(response.get("body").read())

# Extract only the 'text' value
extracted_text = response_body.get("content")[0].get('text')
print(extracted_text)

input_token_count = response["ResponseMetadata"]["HTTPHeaders"][
    "x-amzn-bedrock-input-token-count"
]
output_token_count = response["ResponseMetadata"]["HTTPHeaders"][
    "x-amzn-bedrock-output-token-count"
]

print(f"Input token count: {input_token_count}")
print(f"Output token count: {output_token_count}")


Hello! How can I assist you today? Feel free to ask me any questions or let me know if there's anything you'd like to chat about.
Input token count: 10
Output token count: 34


# 미리디 테스트

## Initial prompt

In [4]:
# Intial Prompt
import boto3
import json

# Define the prompts
SYSTEM_PROMPT = "You need to create a Powerpoint deck in English."
USER_PROMPT = "Paris 2024 Olympics Closing Ceremony Success Prediction"

# Combine system prompt and user prompt
COMBINED_PROMPT = f"{SYSTEM_PROMPT}\n\nTask: {USER_PROMPT}"

bedrock = boto3.client(service_name="bedrock-runtime")

body = json.dumps({
    "max_tokens": 4196,
    "messages": [
        {"role": "user", "content": COMBINED_PROMPT}
    ],
    "anthropic_version": "bedrock-2023-05-31"
})

response = bedrock.invoke_model(body=body, modelId="anthropic.claude-3-5-sonnet-20240620-v1:0")

response_body = json.loads(response.get("body").read())

# Extract only the 'text' value
extracted_text = response_body.get("content")[0].get('text')
print("Claude's response:")
print(extracted_text)

input_token_count = response["ResponseMetadata"]["HTTPHeaders"][
    "x-amzn-bedrock-input-token-count"
]
output_token_count = response["ResponseMetadata"]["HTTPHeaders"][
    "x-amzn-bedrock-output-token-count"
]

print(f"\nInput token count: {input_token_count}")
print(f"Output token count: {output_token_count}")


Claude's response:
Certainly! I'd be happy to help you create a PowerPoint deck outline for predicting the success of the Paris 2024 Olympics Closing Ceremony. Here's a suggested structure for your presentation:

Slide 1: Title Slide
- Title: "Paris 2024 Olympics Closing Ceremony: Success Prediction"
- Subtitle: "An analysis of factors influencing the event's potential success"

Slide 2: Introduction
- Brief overview of the importance of Olympic closing ceremonies
- Mention of Paris as the host city

Slide 3: Key Success Factors
- List of factors that will be analyzed:
  1. Planning and Organization
  2. Cultural Representation
  3. Technological Integration
  4. Athlete and Spectator Experience
  5. Global Audience Engagement

Slide 4: Planning and Organization
- Timeline of preparation
- Involvement of experienced event planners
- Coordination with local authorities and stakeholders

Slide 5: Cultural Representation
- Showcase of French culture and heritage
- Integration of global cu

## 시도1: Prompt Chaining

1. 전체 아웃라인을 짜기 (initial prompt)
2. 1번에서 짠 아웃라인에 맞춰서 디테일한 내용을 불렛 포인트로 정리한 슬라이드를 각각 만들기 (slide_prompt)

In [7]:
# Chaining Prompt
# To create a 20-page PowerPoint presentation using chaining prompts, \n
# we'll need to make multiple calls to Claude, each time asking for a specific slide.

import boto3
import json
import time

def invoke_claude(prompt):
    bedrock = boto3.client(service_name="bedrock-runtime")
    body = json.dumps({
        "max_tokens": 4196,
        "messages": [{"role": "user", "content": prompt}],
        "anthropic_version": "bedrock-2023-05-31"
    })
    
    response = bedrock.invoke_model(body=body, modelId="anthropic.claude-3-5-sonnet-20240620-v1:0")
    response_body = json.loads(response.get("body").read())
    output_text = response_body.get("content")[0].get('text')
    
    input_token_count = int(response["ResponseMetadata"]["HTTPHeaders"]["x-amzn-bedrock-input-token-count"])
    output_token_count = int(response["ResponseMetadata"]["HTTPHeaders"]["x-amzn-bedrock-output-token-count"])
    
    return output_text, input_token_count, output_token_count

# Initial prompt
SYSTEM_PROMPT = "You are an expert in creating PowerPoint presentations. Create a detailed outline for a 20-slide presentation in English."
TOPIC = "Paris 2024 Olympics Closing Ceremony Success Prediction"

# Generate overall outline
outline_prompt = f"{SYSTEM_PROMPT}\n\nTopic: {TOPIC}\n\nProvide a high-level outline for a 20-slide presentation, including a title for each slide."
outline, input_tokens, output_tokens = invoke_claude(outline_prompt)
print("Overall Outline:")
print(outline)
print(f"\n Input tokens: {input_tokens}, Output tokens: {output_tokens}")
print("\n" + "="*50 + "\n")

total_input_tokens = input_tokens
total_output_tokens = output_tokens

# time.sleep(120)

# Generate detailed content for each slide
slides = []
for i in range(1, 21):
    slide_prompt = f"""Based on the following outline for a presentation on {TOPIC}, create detailed content for slide {i}. Include bullet points and any relevant information that should be on this slide.

Outline:
{outline}

Now, provide detailed content for Slide {i}:"""
    
    slide_content, input_tokens, output_tokens = invoke_claude(slide_prompt)
    slides.append(f"Slide {i}:\n{slide_content}")
    print(f"Generated content for Slide {i}")
    print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
    
    total_input_tokens += input_tokens
    total_output_tokens += output_tokens
    
    # time.sleep(120)  # Add a small delay to avoid rate limiting

# Print all slides
print("\n Detailed Slide Contents:")
for slide in slides:
    print(slide)
    print("\n" + "-"*50 + "\n")

print("PowerPoint presentation generation complete!")
print(f"Total Input Tokens: {total_input_tokens}")
print(f"Total Output Tokens: {total_output_tokens}")


Overall Outline:
Here's a detailed outline for a 20-slide presentation on "Paris 2024 Olympics Closing Ceremony Success Prediction":

1. Title Slide: Paris 2024 Olympics Closing Ceremony Success Prediction

2. Introduction: Overview of the Paris 2024 Olympics

3. Historical Context: Past Olympic Closing Ceremonies

4. Paris 2024 Vision: Goals and Themes

5. Venue: Seine River and Trocadéro

6. Technological Innovations: Planned Digital Enhancements

7. Cultural Showcase: Highlighting French Heritage

8. Artistic Direction: Key Creative Minds Behind the Ceremony

9. Music and Performance: Expected Artists and Genres

10. Athlete Participation: Roles and Involvement

11. Sustainability Initiatives: Eco-friendly Aspects of the Ceremony

12. Logistical Challenges: Transportation and Crowd Management

13. Security Measures: Ensuring Safety for All Attendees

14. Global Broadcast: Expected Viewership and Coverage

15. Social Media Impact: Predicted Engagement and Trends

16. Economic Impact:

## 시도2: Zero shot prompting 이지만 Role-based prompting, Task-specific instructions, Content structuring, Output constraints를 사용
: Structure the presentation을 미리 prompt (presentation_prompt) 에 써주고, 각 슬라이드의 아웃풋 토큰 개수에 제한을 두기

In [ ]:
# Reduce the number of API calls by generating content for multiple slides in a single call. \n
# This approach will be more efficient and can potentially provide better coherence across slides.
import boto3
import json

def invoke_claude(prompt):
    bedrock = boto3.client(service_name="bedrock-runtime")
    body = json.dumps({
        "max_tokens": 4196,
        "messages": [{"role": "user", "content": prompt}],
        "anthropic_version": "bedrock-2023-05-31"
    })
    
    response = bedrock.invoke_model(body=body, modelId="anthropic.claude-3-5-sonnet-20240620-v1:0")
    response_body = json.loads(response.get("body").read())
    return response_body.get("content")[0].get('text'), int(response["ResponseMetadata"]["HTTPHeaders"]["x-amzn-bedrock-output-token-count"])

TOPIC = "Paris 2024 Olympics Closing Ceremony Success Prediction"

# Generate entire presentation in one call
presentation_prompt = f"""As an expert in creating PowerPoint presentations, create a detailed 20-slide presentation in English on the topic: {TOPIC}

For each slide, provide:
1. A clear and concise title
2. 3-5 bullet points of key information
3. Any relevant data or statistics (if applicable)

Structure the presentation as follows:
- Slide 1: Introduction
- Slides 2-3: Background and Context
- Slides 4-7: Main Points and Analysis
- Slides 8-9: Predictions and Potential Outcomes
- Slide 10: Conclusion and Key Takeaways

Present the content for all 20 slides in a clear, organized manner. The number of output tokens for each slide must not exceed 100."""

presentation_content, total_output_tokens = invoke_claude(presentation_prompt)

print("Generated Presentation Content:")
print(presentation_content)
print("\n" + "="*50 + "\n")
print(f"Total output tokens: {total_output_tokens}")
print("PowerPoint presentation generation complete!")

# Optional: Split the content into individual slides
slides = presentation_content.split("Slide")[1:]  # Split by "Slide" and remove empty first element
slides = ["Slide" + slide for slide in slides]  # Add "Slide" back to each element

print("\nIndividual Slides:")
for slide in slides:
    print(slide.strip())
    print("\n" + "-"*50 + "\n")


Generated Presentation Content:
Here's a detailed 20-slide presentation on "Paris 2024 Olympics Closing Ceremony Success Prediction":

Slide 1: Introduction
Title: Paris 2024 Olympics Closing Ceremony: Predicting Success
• Overview of the presentation
• Importance of the closing ceremony
• Key factors influencing success prediction

Slide 2: Background - Paris 2024 Olympics
Title: Paris 2024 Olympics: An Overview
• Dates: July 26 - August 11, 2024
• 33 sports, 329 events
• Expected participation: 10,500 athletes
• 206 National Olympic Committees

Slide 3: Context - Olympic Closing Ceremonies
Title: Olympic Closing Ceremonies: A Tradition
• Purpose and significance
• Historical highlights
• Evolution of closing ceremonies
• Importance for host cities

Slide 4: Main Point 1 - French Cultural Showcase
Title: Celebrating French Culture and Heritage
• Integration of iconic French symbols
• Showcase of art, music, and dance
• Potential celebrity appearances
• Highlighting Parisian landmarks
